#Main

In [52]:
def main():
    import os
    import shutil
    import pandas as pd
    # from openpyxl.workbook import Workbook
    # from openpyxl import load_workbook
    import numpy as np
    # import matplotlib.pyplot as plt
    import math
    import os
    import re
    import random
    import time
    # from fpdf import FPDF
    from datetime import date
    from fractions import Fraction
    import pickle 
    
    exe_directory = os. getcwd()
    directory = os.path.join(exe_directory,"Zero Waste Orders") 
    init = title_frame()
    if init == True:
        #Make directory
        if os.path.exists(directory):
            shutil.rmtree(directory)
        os.mkdir(directory)
        
        inventory = pd.DataFrame(columns={'ingredient':[],'qty':[],'unit':[],'shelf_life':[]})
        inventory.to_csv(os.path.join(directory,"inventory.csv"), index=False)

        ordered_inventory = pd.DataFrame(columns={'ingredient':[],'qty':[],'unit':[],'shelf_life':[]})
        ordered_inventory.to_csv(os.path.join(directory,"pending_inventory.csv"), index=False)

        recipe_made = pd.DataFrame(columns={'day':[],'recipe':[],'qty':[]})
        recipe_made.to_csv(os.path.join(directory,"pending_inventory.csv"), index=False)
        
        #initialization prompts
        user_recipe_directory = recipe_directory_frame()
        init_inventory_path = init_inventory_frame()
        recipe_demand_path = recipe_demand_frame()
        current_order_time = current_order_time_frame()
        #values to obtain from future prompts
        order_forecast_days = 7
        
        #store directories
        directory_mapping = {
        'user_recipe_directory': user_recipe_directory,
        'init_inventory_path': init_inventory_path,
        'recipe_demand_path': recipe_demand_path
        }

        with open(os.path.join(directory,'directory_mapping.txt'), 'w') as file:
            for var_name, directory in directory_mapping.items():
                file.write(f"{var_name}={directory}\n")
        
    if init == False:
        #recall directories
        directory_mapping = {}
        with open(os.path.join(directory,'directory_mapping.txt'), 'r') as file:
            for line in file:
                var_name, directory = line.strip().split('=')
                directory_mapping[var_name] = directory

        if 'user_recipe_directory' in directory_mapping:
            user_recipe_directory = directory_mapping['user_recipe_directory']

        if 'init_inventory_path' in directory_mapping:
            init_inventory_path = directory_mapping['init_inventory_path']

        if 'recipe_demand_path' in directory_mapping:
            recipe_demand_path = directory_mapping['recipe_demand_path']
        
    #app prompts
    menu_selection = menu_frame()
    print("menu_Selection:",menu_selection)
    if menu_selection == 'update_inventory':
        inventory_update_frame()
        exit_frame()
    elif menu_selection == 'exit':
        exit_frame()
    elif menu_selection == 'stock_inventory':
        #inventory_stock_frame()
        exit_frame()
        
#         # create grocery list
#         initilize(recipe_directories)
#         estimateDailyUsage(recipe_directories)

        
#     #from other code
# #     ordered_inventory = pd.DataFrame({'ingredient': ['onion', 'potato'],
# #                            'qty': [2, 10],
# #                            'unit': ['tsp','kg'],
# #                            'shelf_life': [1,2]})
# #     recipe_made = pd.DataFrame({'item': ['Naan','Naan'],
# #                             'qty': [1,2]})
# #     isInventory_stocked = True
    
    
#     order_forecast_days = 7
#     #### Function Calls
    
# #     inventoryManager(directory, recipe_directories, ordered_inventory, isInventory_stocked, recipe_made)

#     # create grocery list
    
#     initilize(recipe_directories)
#     estimateDailyUsage(recipe_directories)

    recipe_made = pd.DataFrame({'item': ['Bombay Aloo','Naan'],'qty': [1,2]})
    isInventory_stocked = True
    order_forecast_days = 7
    
    #Variable Initialization
    user_recipe_directory_list = os.listdir(user_recipe_directory)
    recipe_directories = [os.path.join(user_recipe_directory, item) for item in user_recipe_directory_list]
    
    #### Function Calls
    
    inventoryManager(exe_directory, recipe_directories, isInventory_stocked, recipe_made)

    # create grocery list
    
    initilize(recipe_directories)
    daily_usage_df = estimateDailyUsage(recipe_directories)
    grocery_list_df = createGroceryList(exe_directory, daily_usage_df, recipe_directories, order_forecast_days)
    print(grocery_list_df)
        

In [38]:
# #mock input from init 
# import pandas as pd
# inventory = pd.DataFrame({'ingredient': ['onion', 'carrot'],
#                            'qty': [1, 2],
#                            'unit': ['tsp','tsp'],
#                            'shelf_life': [1,2]})
# ordered_inventory = pd.DataFrame({'ingredient': ['onion', 'potato'],
#                            'qty': [2, 10],
#                            'unit': ['tsp','kg'],
#                            'shelf_life': [1,2]})
# recipe_made = pd.DataFrame({'item': ['Naan','Naan'],
#                             'qty': [1,2]})
# isInventory_stocked = True

In [39]:
def inventoryManager(directory, recipe_directories, isInventory_stocked, recipe_made):  
    import pandas as pd
    import os
    
    inventory_path = os.path.join(directory,"inventory.csv")
    ordered_inventory_path = os.path.join(directory,"ordered_inventory.csv")
    inventory = pd.read_csv(inventory_path)
    
    ''''
    if isInventory_stocked == True:
        #stock inventory
        for index, row in ordered_inventory.iterrows():
            ingredient = row['ingredient']
            matching_row = inventory[inventory['ingredient'] == ingredient]

            if not matching_row.empty:
                inventory.loc[matching_row.index, 'qty'] += row['qty']
                inventory.loc[matching_row.index, 'shelf_life'] += row['shelf_life']
                inventory.loc[matching_row.index, 'unit'] = row['unit']
            
            ordered_inventory = pd.DataFrame(columns={'ingredient':[],'qty':[],'unit':[],'shelf_life':[]})
            ordered_inventory.to_csv(os.path.join(directory,"pending_inventory.csv"), index=False)
    '''
    times_made = recipe_made
    for recipe in recipe_directories:
        recipe_df = pd.read_csv(recipe)
        # Get the number of times the recipe was made
        recipe_name = os.path.basename(recipe)# Extract the recipe name from the file path
        recipe_name = recipe_name.split('.csv')
        recipe_name = recipe_name[0]
        #num_times_made = times_made.get('qty')
        if recipe_name in times_made['item'].tolist():
            times_made_idx = times_made[times_made['item']==recipe_name].index[0]
            num_times_made = times_made['qty'].loc[times_made_idx]

            # Iterate through each row in the recipe DataFrame
            for index, row in recipe_df.iterrows():
                ingredient = row['ingredient']
                qty = row['qty'] * num_times_made
                unit = row['unit']
                component = row['component']
                used_inventory = pd.DataFrame({'ingredient': [],'qty': [],'unit': [],'component': []})
                #used_inventory = used_inventory.append(new_row, ignore_index=True)
                

                # Check if the ingredient is already in used_inventory
                if used_inventory.empty == False:
                    existing_row = used_inventory[used_inventory['ingredient'] == ingredient]

                    if existing_row.empty == False:
                        used_inventory.loc[existing_row.index, 'qty'] += qty
                    else:
                        new_row_df = pd.DataFrame({
                            'ingredient': [ingredient],
                            'qty': [qty],
                            'unit': [unit],
                            'component': [component]
                        })
                        used_inventory = used_inventory.append(new_row_df, ignore_index=True)
                        
                else:
                    new_row_df = pd.DataFrame({
                        'ingredient': [ingredient],
                        'qty': [qty],
                        'unit': [unit],
                        'component': [component]
                    })
                    used_inventory = used_inventory.append(new_row_df, ignore_index=True)

        #remove inventory        
        
            for index_rec, row_rec in recipe_df.iterrows():
                ingredient_rec = row_rec['ingredient']
                qty_rec = row_rec['qty']
                ingredient = row['ingredient']
                matching_row = inventory[inventory['ingredient'] == ingredient_rec]
                if matching_row.empty == False:
                    inventory.loc[matching_row.index, 'qty'] -= qty_rec*num_times_made
                inventory.to_csv(os.path.join(directory,"inventory.csv"), index=False)
            #note inventory to throw away

        #store inventory
    inventory.to_csv(inventory_path,index=False)

    
    from datetime import datetime
    from threading import Timer
    
    #end of each day, subtract 1 from shelf life for corrent inventory
    x=datetime.today()
    y=x.replace(day=x.day +1, hour=0, minute=0, second=0, microsecond=0)
    delta_t=y-x

    secs=delta_t.seconds+1

    def shelf_life_update():
        inventory['shelf_life'] -= 1
        inventory.to_csv(inventory_path,index=False)
        

    t = Timer(secs, shelf_life_update())
    t.start()
    
    
    #remove expired inventory
    expired_rows = inventory[inventory['shelf_life'] == 0]
    expired_inventory = expired_rows.copy()
    inventory = inventory[inventory['shelf_life'] > 0]
    if not expired_inventory.empty:
        print("Expired Inventory:")
        print(expired_inventory)
    
    #store inventory
    inventory.to_csv(inventory_path,index=False)
    
    
    
    
    
#     #add to inventory
#     def add_item(item_name, quantity):
#     if item_name in shopping_list.keys():
#         shopping_list[item_name] += quantity
#     else:
#         shopping_list[item_name] = quantity

#     add_item("Bread", 1)
    

In [40]:
def initilize(recipe_directories):
#     #Run through the directory and compile all recipes_df dataframes into a list recipes_df_list.
#     recipe_df_list = pd.DataFrame()
#     for recipe in recipe_directories:
#         recipe_df = pd.read_csv(recipe)
#         print(recipe_df)
#         recipe_df_list = recipe_df_list.append(recipe_df, ignore_index=True)
#         print(recipe_df_list)
    import pandas as pd
    import os
    
    #From that create a master list of all ingredients used and their replenishament as a user defined input.
    ingredient_breakdown_directory = "D:\\AI_ML_DL\\Hackathon\\Codes"
    ingredients_master_df = pd.read_csv(os.path.join(ingredient_breakdown_directory,"ingredient_breakdown.csv"))
    ingredients_master_df = ingredients_master_df.sort_values(by='ingredient', ascending=True)
    ingredients_master_df = ingredients_master_df.reset_index(drop=True)

    return ingredients_master_df

In [41]:
def estimateDailyUsage(user_recipe_directory):
#     #Run through the directory and compile all recipes_df dataframes into a list recipes_df_list.
    
#     #Consider the receipes_demand dataframe and then compute average weekly demand (sum each recipe's daily demand and divide by 7)
#     # and make a temp_df with it (#recipes = #rows of the temp_df)
#     temp_df = pd.DataFrame({recipe:[Bombay Aloo, Chicken Tikka Masala, Naan, Onion Bhaji, Pilau Rice, Plain Papadum],
#                             demand:[3, 5, 1, 2, 6, 1]})
#     #Now based on the temp_df rows i.e. each recipe get the list of ingredients and quantity from the recipe files.
#     # - For that match the name of the recipe in the temp_df and add '_df' to match with the element in the recipes_df_list
    
#     # - Create a final_df with the ingredients with its respective quantities multiplied by the demand 
#     # - For the next recipe, repeat the first two steps
#     # - -and then check if ingredient is already present in the df; if yes, sum up the quantity (quantity for that recipe*demand)
#     # - - -if not, create a new row
#     # - If the ingredients are present in different quantities in different recipes, consider different rows (for now)
#     # - Save the final_df as daily_usage_df.
    
#     daily_usage_df = pd.DataFrame({'ingredient': [],
#                                        'qty': [],
#                                        'unit': [],
#                                        'replenish_value': []})
    import os
    import pandas as pd
    # Step 1: Run through the directory and compile all recipe_df DataFrames into a list recipe_df_list.
    recipe_df_list = []
    recipe_name_list = []
    recipe_master_df = pd.DataFrame(columns=['recipe_name','recipe_df'])
    for recipe in recipe_directories:
        recipe_df = pd.read_csv(recipe)
        recipe_df_list.append(recipe_df)
        recipe_name = os.path.basename(recipe)# Extract the recipe name from the file path
        recipe_name = recipe_name.split('.csv')
        recipe_name_list.append(recipe_name[0])
        recipe_master_df = recipe_master_df.append({'recipe_name': recipe_name[0], 'recipe_df': recipe_df}, ignore_index =True)
    
    # Step 2: Compute average weekly demand and create a temp_df
    recipes = ['Bombay Aloo', 'Chicken Tikka Masala', 'Naan', 'Onion Bhaji', 'Pilau Rice', 'Plain Papadum']
    demand = [3, 5, 1, 2, 6, 1]

    temp_df = pd.DataFrame({'recipe': recipes, 'demand': demand})

    # Step 3: Iterate through the temp_df and match recipe names with recipe_df_list
    final_df = pd.DataFrame(columns=['ingredient', 'quantity'])

    for index, row in temp_df.iterrows():
        recipe_name = row['recipe']
        recipe_demand = row['demand']
        recipe_df = None
        for df in recipe_master_df['recipe_df'].tolist():
            if recipe_name in recipe_master_df['recipe_name'].values:
                idx = recipe_master_df[recipe_master_df['recipe_name']==recipe_name].index[0]
                recipe_df = recipe_master_df['recipe_df'][idx]
                break

        if recipe_df is not None:
            for _, recipe_row in recipe_df.iterrows():
                ingredient = recipe_row['ingredient']
                quantity = recipe_row['qty']

                existing_row = final_df[final_df['ingredient'] == ingredient]
                if not existing_row.empty:
                    final_df.loc[existing_row.index, 'quantity'] += quantity * recipe_demand
                else:
                    final_df = final_df.append({'ingredient': ingredient, 'quantity': quantity * recipe_demand}, ignore_index=True)

    # Step 4: Save the final_df as daily_usage_df
    daily_usage_df = final_df.copy()
    daily_usage_df = daily_usage_df.sort_values(by='ingredient', ascending=True)
    daily_usage_df = daily_usage_df.reset_index(drop=True)
    # Display the resulting daily_usage_df

    return daily_usage_df

In [42]:
def createGroceryList(directory, daily_usage_df, recipe_directories, order_forecast_days):
    
    import pandas as pd
    import os
    from datetime import datetime
    from threading import Timer

    inventory_path = os.path.join(directory,"inventory.csv")
    ordered_inventory_path = os.path.join(directory,"ordered_inventory.csv")
    inventory_df = pd.read_csv(inventory_path)
    ingredients_master_df = initilize(recipe_directories)
        
    # use order_forcast_days to determine qty_required
    #Create an empty dataframe 'grocery_list_df' with columns 'ingredient' and 'qty_required'
    grocery_list_df = pd.DataFrame(columns=['ingredient','qty'])
    
    #1. Consider the 'inventory_df'
    # - check if all ingredients from the ingredients_master_df are present in inventory df:
    # - - if not: add rows: ingd, qty = 0, unit, shelf_life = replenishment_value
    ingredients_list = ingredients_master_df['ingredient'].tolist()
    unit_list = ingredients_master_df['unit'].tolist()
    replenish_list = ingredients_master_df['replenish_value'].tolist()
    for i in range(len(ingredients_list)):
        if ingredients_list[i] not in inventory_df['ingredient'].tolist():
            inventory_df.append({"ingredient":ingredients_list[i], "qty":0.0, "unit":unit_list[i], 
                                "shelf_life": replenish_list[i]}, ignore_index=True)
    
    #2. Start a for loop for each ingredient:
#     for i in range(len(inventory_df)):
#         ingredient_name = inventory_df['ingredient'][i]
#         replenish_value_idx = ingredients_master_df[ingredients_master_df['ingredient'] == ingredient_name].index[0]
#         replenish_value = ingredients_master_df['replenish_value'][replenish_value_idx]
#         shelf_life = inventory_df['shelf_life'][i]
#         present_qty = inventory_df['qty'][i]
        
#         dailyuse_idx = daily_usage_df[daily_usage_df['ingredient'] == ingredient_name].index[0]
#         daily_use = daily_usage_df['quantity'][dailyuse_idx]
        
#         if (order_forecast_days <= replenish_value) & (order_forecast_days <= shelf_life):
#             qty_reqd = daily_use*order_forecast_days - present_qty
#         elif (order_forecast_days <= replenish_value) & (order_forecast_days > shelf_life):
#             qty_reqd = daily_use*order_forecast_days - present_qty
#         else:
#             qty_reqd = daily_use*replenish_value - present_qty
        
#         grocery_list_df = grocery_list_df.append({'ingredient': ingredient_name, 'qty' : qty_reqd}, ignore_index = True)
    #Sum up all the common ingredient rows
    for i in range(len(ingredients_master_df)):
        ingredient_name = ingredients_master_df['ingredient'][i]
        print(ingredient_name)
        replenish_value = ingredients_master_df['replenish_value'][i]
        dailyuse_idx = daily_usage_df[daily_usage_df['ingredient'] == ingredient_name].index[0]
        daily_use = daily_usage_df['quantity'][dailyuse_idx]
        try:
            if (ingredient_name in inventory_df['ingredient'].tolist()) == True:
                shelf_life = inventory_df['shelf_life'][i]
                present_qty = inventory_df['qty'][i]

                if (order_forecast_days <= replenish_value) & (order_forecast_days <= shelf_life):
                    qty_reqd = abs(daily_use*order_forecast_days - present_qty)
                elif (order_forecast_days <= replenish_value) & (order_forecast_days > shelf_life):
                    qty_reqd = abs(daily_use*order_forecast_days - present_qty)
                else:
                    qty_reqd = abs(daily_use*replenish_value - present_qty)
            else:
                qty_reqd = abs(daily_use*replenish_value)

            grocery_list_df = grocery_list_df.append({'ingredient': ingredient_name, 'qty' : qty_reqd}, ignore_index = True)
            grocery_list_df = grocery_list_df.groupby('ingredient', as_index=False)['qty'].sum()
        except:
            continue
    grocery_list_df.to_csv(os.path.join(directory,"pending_inventory.csv"), index=False)
    #grocery list should only have ingredient and quantity in the column
    
    return grocery_list_df

In [46]:
# -*- coding: utf-8 -*-
"""
Created on Sat Aug 12 14:12:38 2023

@author: Ainsley

"""
import tkinter as tk
from tkinter import filedialog as fd
from tkinter import *
from tkinter import Tk, simpledialog

'''
# Here are two sample segments of code

#This code will store the filepath selected by a user from the tkinter input box
box_1 = tk.Tk()
box_1.title("TextBox Input")
box_1.geometry('400x200')
#pop-up box will ask for filename
def ask_filename():
    global filename
    filename = fd.askopenfilename()
errmsg = 'Error!'
tk.Button(text='Click to Open File',
       command=ask_filename).pack(fill=tk.X)
box_1.mainloop()

#This code will store the folder selected by a user from the tkinter input box
box_2 = tk.Tk()
box_2.title("TextBox Input")
box_2.geometry('400x200')
def ask_folder():
    global directory
    directory = fd.askdirectory()
errmsg = 'Error!'
tk.Button(text='Click to Open Folder',
       command=ask_folder).pack(fill=tk.X)

box_2.mainloop()
'''
def title_frame():
    box_1 = tk.Tk()
    box_1.title("Select User Type")
    l_1 = Label(box_1, text = "Please close the window once you have made your selection.")
    box_1.geometry('5000x4000')
    def new_user():
            global init
            init = True

    def login():
            global init
            init = False

    tk.Button(text='New User',
           command=new_user).pack(fill=tk.X)
    tk.Button(text='Login',
           command=login).pack(fill=tk.X)
    l_1.pack()
    box_1.mainloop()
    return init

#if the user information needs to be initialized
# if new_user:
def recipe_directory_frame():
    #pop up box will ask for folder of recipes
    box_2 = tk.Tk()
    box_2.title("Select the folder that contains the recipes.")
    box_2.geometry('5000x4000')
    def ask_folder_recipe():
        global user_recipe_directory
        user_recipe_directory = fd.askdirectory()
    errmsg = 'Error!'
    tk.Button(text='Click to Open Folder',
           command=ask_folder_recipe).pack(fill=tk.X)
    l_2 = Label(box_2, text = "Please close the window once you have made your selection.")
    l_2.pack()
    box_2.mainloop()
    return user_recipe_directory
    
    #pop-up will ask for current inventory
def init_inventory_frame():
    box_3 = tk.Tk()
    box_3.title("Select the file that contains the current inventory information.")
    box_3.geometry('5000x4000')
    #pop-up box will ask for filename
    def ask_inventory_filename():
        global init_inventory_directory
        init_inventory_directory = fd.askopenfilename()
        errmsg = 'Error!'
    tk.Button(text='Click to Open File',
                  command=ask_inventory_filename).pack(fill=tk.X)
    l_3 = Label(box_3, text = "Please close the window once you have made your selection.")
    l_3.pack()
    box_3.mainloop()
    return init_inventory_directory
    
    #pop-up will ask for current inventory
def recipe_demand_frame():
    box_4 = tk.Tk()
    box_4.title("Select the file that contains the customer order history data.")
    box_4.geometry('5000x4000')
    #pop-up box will ask for filename
    def ask_order_history_filename():
        global recipe_demand_path
        recipe_demand_path = fd.askopenfilename()
        errmsg = 'Error!'
    tk.Button(text='Click to Open File',
                  command=ask_order_history_filename).pack(fill=tk.X)
    l_4 = Label(box_4, text = "Please close the window once you have made your selection.")
    l_4.pack()
    box_4.mainloop()
    return recipe_demand_path

def current_order_time_frame():
    box_5 = tk.Tk()
    #box_5.attributes("-fullscreen", True)
    box_6.geometry('5000x4000')
    box_5.withdraw()
    current_order_time = simpledialog.askinteger(title="Ordering Time", prompt="Please enter the amount of time you previously spent on ordering in hours")
    box_5.destroy()
    return current_order_time

#if the user information does not need to be initialized
# elif not new_user:

def menu_frame():
    box_6 = tk.Tk()
    box_6.title("Select User Type")
    l_6 = Label(box_6, text = "Please close the window once you have made your selection.")
    box_6.geometry('5000x4000')
    def update_inventory():
            global menu_selection
            menu_selection = 'update_inventory'
            
    def exit():
            global menu_selection
            menu_selection = 'exit'

    tk.Button(text='Update inventory',
           command=update_inventory).pack(fill=tk.X)
    tk.Button(text='Exit',
           command=exit).pack(fill=tk.X)
    l_6.pack()
    box_6.mainloop()
    return menu_selection
        
#     if update_inventory:
def inventory_update_frame():      
    box_7 = tk.Tk()
    box_7.title("Inventory Update")
    box_7.geometry('5000x4000')
    l_7 = Label(box_7, text = "A copy of the current inventory status has been created in your directory. Please correct the current inventory counts as directed and re-upload.")
    #pop-up box will ask for filename
    def ask_updated_inventory_filename():
        global updated_inventory_filename
        updated_inventory_filename = fd.askopenfilename()
    errmsg = 'Error!'
    tk.Button(text='Click to Open File',
           command=ask_updated_inventory_filename).pack(fill=tk.X)
    l_7.pack()
    box_7.mainloop()
    return

    #elif not update_inventory:
def exit_frame():
    box_8 = tk.Tk()
    box_8.title("Inventory Update Not Needed")
    box_8.geometry('5000x4000')
    #remove go-away
    l_8 = Label(box_8, text = "Thank you for using Zero Order Waste!")
    l_8.pack()
    box_8.mainloop()
    return


In [47]:
# # -*- coding: utf-8 -*-
# """
# Created on Sat Aug 12 08:20:13 2023

# @author: Ainsley

# this program proccesses the input data collected by the tkinter modules the user can itneract with
# """

# import pandas as pd
# import numpy as np
# from os import listdir

# #this function could collect the input data from the tkinter input boxes to the larger operation of the program. 
# #The variable names currently do not match, but could be edited 
# def collect_input_data():
# #list of dishes and recipies
#     list_of_recipies = listdir()

# #ingredient list
#     df_inventory_list = pd.read_csv(inventory_list_path)

# #ingredient cost
#     df_ingredient_cost = pd.read_csv(ingredient_cost_path)

# #shelf life
#     df_shelf_life =pd.read_csv(shelf_life_path)
#     shelf_life_dict = {df_shelf_life['Ingredient'][i]: df_shelf_life['Shelf Life'][i][i] for i in range(len(df_shelf_life))}

# #recipies made
#     df_recipies_made = pd.read_csv(recipies_made_path)

# #recipe demand
#     df_recipies_demand = generate_recipe_demand(df_recipies_made)
        
# #customer traffic
#     df_customer_traffic = pd.read_csv(customer_traffic_path)

# #current ordering time

# #unit conversion dictionary to change from recipe units to the units we will likely see related to cost from suppliers
#     #unit : [unit to ml, unit to g]
#     unit_conversion_dict = {'cup' :[237	,240] , 'tbsp':[14.7867,	14.7867] , 'tsp':[4.9289	,4.9289]}

#     return 

# #this function generates a dataframe relating the number of recipies made and the day of the week to the predicted demand of a recipe.
# def generate_recipe_demand(df_recipies_made,list_of_recipies):
#     df_recipies_demand = pd.DataFrame((np.zeros(7*len(list_of_recipies),3)), columns = ['Recipe','Average Quantity Per Day', 'Day'])
#     row = 0
#     for recipe in list(df_recipies_made.unique()):
#         for day in range(1,8):
#             df_date_recipe_subset = df_recipies_made[df_recipies_made['Recipe'] == recipe]
#             df_date_recipe_subset = df_date_recipe_subset[df_date_recipe_subset['Day'] == day]
#             list_date_recipe_subset = list(df_date_recipe_subset['Quantity'])
#             number_of_recipe_ordered = sum(list_date_recipe_subset)/len(list_date_recipe_subset)
#             df_recipies_demand['Recipe'][row] = recipe
#             df_recipies_demand['Average Quantity'][row] = number_of_recipe_ordered
#             df_recipies_demand['Day'][row] = day
#             row = row + 1
#     return df_recipies_demand
        

In [54]:
main()

-----------------------------------------------------
-----------------------------------------------------
-----------------------------------------------------
-----------------------------------------------------
-----------------------------------------------------
-----------------------------------------------------
-----------------------------------------------------
-----------------------------------------------------
-----------------------------------------------------
-----------------------------------------------------
-----------------------------------------------------
-----------------------------------------------------
-----------------------------------------------------
-----------------------------------------------------


NameError: name 'box_6' is not defined